<h1>Example post quantum secure communication between ALice and Bob</h1>

1. Required prior knowledge of the buffer size used, for all the parties.

In [21]:
buffer_size = 640 # y of (y X y)
sha_key_str = 'Tuesday Evening'

2. QuantCrypt provides Kyber1024 key encapsulation mechanism.

In [22]:
import oqs

kem = "ML-KEM-1024"

3. Alice generates key pair.

In [23]:
alice = oqs.KeyEncapsulation(kem)
public_key_alice = alice.generate_keypair()

4. Bob generates cipher and shared secret using Alice's public key.

In [24]:
bob = oqs.KeyEncapsulation(kem)
cipher_text_bob, shared_secret = bob.encap_secret(public_key_alice)


5. Alice decrypts cipher she receives from Bob to extract shared secret.

In [25]:

shared_secret_copy = alice.decap_secret(cipher_text_bob)


6. Bob encrypts the message to be sent using the shared_secret.

In [26]:
from selective_encryption import encrypt
from utils import time_it

encrypt(file_path='data/files/32MB_image.png',
           sha_key_str=sha_key_str, 
           buffer=buffer_size,
           aes_key=shared_secret,
           save_data=True)

itr_count: 0
_get_2d_array: 0.000020666
dwt2d: 0.002703125
dwt2d: 0.000628542
sha256_with_key: 0.000019459
sha512_with_key: 0.000055000
xor_with_sha_key_numba: 0.000075208
xor_with_sha_key: 0.000090333
xor_with_sha_key_numba: 0.000277000
xor_with_sha_key: 0.000286625
aes_encrypt: 0.000144334
_se_encrypt: 0.004302541
itr_count: 1
_get_2d_array: 0.000007250
dwt2d: 0.000793041
dwt2d: 0.001756875
sha256_with_key: 0.000043500
sha512_with_key: 0.000075750
xor_with_sha_key_numba: 0.000303958
xor_with_sha_key: 0.000333666
xor_with_sha_key_numba: 0.001823458
xor_with_sha_key: 0.001881042
aes_encrypt: 0.000280833
_se_encrypt: 0.005952833
itr_count: 2
_get_2d_array: 0.000057291
dwt2d: 0.002348667
dwt2d: 0.000673667
sha256_with_key: 0.000028458
sha512_with_key: 0.000082917
xor_with_sha_key_numba: 0.000428250
xor_with_sha_key: 0.000470500
xor_with_sha_key_numba: 0.001011875
xor_with_sha_key: 0.001060750
aes_encrypt: 0.000273209
_se_encrypt: 0.005577333
itr_count: 3
_get_2d_array: 0.000044583
dwt2d:

7. Alice decrypts the message using shared_secret_copy.

In [27]:
from selective_encryption import decrypt

decrypt(file_PF= 'data/files/enc/32MB_image.png_PF.npy',
           file_PPF_1= 'data/files/enc/32MB_image.png_PPF_1.npy',
           file_PPF_2= 'data/files/enc/32MB_image.png_PPF_2.npy',
           buffer=buffer_size,
           output_file_path='data/files/dec/32MB_image.png',
           sha_key_str=sha_key_str, 
           aes_key=shared_secret_copy)

aes_decrypt: 0.000123333
sha256_with_key: 0.000123625
xor_with_sha_key_numba: 0.000079083
xor_with_sha_key: 0.000087459
sha512_with_key: 0.000052625
xor_with_sha_key_numba: 0.000294959
xor_with_sha_key: 0.000304167
idwt2d: 0.000562958
idwt2d: 0.000988333
aes_decrypt: 0.000139167
sha256_with_key: 0.000017000
xor_with_sha_key_numba: 0.000238083
xor_with_sha_key: 0.000246209
sha512_with_key: 0.000056792
xor_with_sha_key_numba: 0.000310625
xor_with_sha_key: 0.000324375
idwt2d: 0.000511959
idwt2d: 0.000999875
aes_decrypt: 0.000114125
sha256_with_key: 0.000014750
xor_with_sha_key_numba: 0.000168208
xor_with_sha_key: 0.000175833
sha512_with_key: 0.000052667
xor_with_sha_key_numba: 0.000292625
xor_with_sha_key: 0.000301459
idwt2d: 0.000494750
idwt2d: 0.001158958
aes_decrypt: 0.000127292
sha256_with_key: 0.000015625
xor_with_sha_key_numba: 0.000084167
xor_with_sha_key: 0.000091750
sha512_with_key: 0.000056667
xor_with_sha_key_numba: 0.000256917
xor_with_sha_key: 0.000264916
idwt2d: 0.000474875
